In [312]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer,LabelEncoder,StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,mean_squared_log_error,roc_curve,precision_recall_curve
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,ExtraTreesRegressor,GradientBoostingRegressor

In [313]:
train1=pd.read_excel(r'C:\Users\Rahul\Desktop\ML\Final_Train.xlsx')
test1=pd.read_excel(r'C:\Users\Rahul\Desktop\ML\Final_Test.xlsx')

In [314]:
train1.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees
0,"BHMS, MD - Homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98%,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250
4,BAMS,20 years experience,100%,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250


In [315]:
test1.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info
0,MBBS,35 years experience,NaN,"Ghatkopar East, Mumbai",General Medicine,NaN
1,"MBBS, Diploma in Otorhinolaryngology (DLO)",31 years experience,NaN,"West Marredpally, Hyderabad",ENT Specialist,NaN
2,"MBBS, DDVL",40 years experience,70%,"KK Nagar, Chennai",Dermatologists,"70% 4 Feedback KK Nagar, Chennai"
3,BAMS,0 years experience,NaN,"New Ashok Nagar, Delhi",Ayurveda,NaN
4,"BDS, MDS - Conservative Dentistry & Endodontics",16 years experience,100%,"Kanakpura Road, Bangalore",Dentist,General Dentistry Conservative Dentistry Cosme...


In [316]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5961 entries, 0 to 5960
Data columns (total 7 columns):
Qualification         5961 non-null object
Experience            5961 non-null object
Rating                2659 non-null object
Place                 5936 non-null object
Profile               5961 non-null object
Miscellaneous_Info    3341 non-null object
Fees                  5961 non-null int64
dtypes: int64(1), object(6)
memory usage: 326.1+ KB


In [317]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987 entries, 0 to 1986
Data columns (total 6 columns):
Qualification         1987 non-null object
Experience            1987 non-null object
Rating                897 non-null object
Place                 1981 non-null object
Profile               1987 non-null object
Miscellaneous_Info    1153 non-null object
dtypes: object(6)
memory usage: 93.2+ KB


In [318]:
train1.describe()

,Fees
count,5961.000000
mean,307.944640
std,190.920373
min,5.000000
25%,150.000000
50%,300.000000
75%,500.000000
max,950.000000


In [319]:
test1.describe()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info
count,1987,1987,897,1981,1987,1153
unique,630,57,36,592,6,1084
top,BAMS,10 years experience,100%,"Mulund West, Mumbai",General Medicine,Acrylic Partial Denture Crowns and Bridges Fix...
freq,153,110,223,27,444,10


In [320]:
train1.isnull().sum()

Qualification            0
Experience               0
Rating                3302
Place                   25
Profile                  0
Miscellaneous_Info    2620
Fees                     0
dtype: int64

In [321]:
test1.isnull().sum()

Qualification            0
Experience               0
Rating                1090
Place                    6
Profile                  0
Miscellaneous_Info     834
dtype: int64

In [322]:
train1['Rating'].replace('(%)','',regex=True,inplace=True)
test1['Rating'].replace('(%)','',regex=True,inplace=True)

In [323]:
train1.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees
0,"BHMS, MD - Homeopathy",24 years experience,100,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250
4,BAMS,20 years experience,100,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250


In [324]:
train1.Profile.value_counts()

Dentist             1397
General Medicine    1173
Dermatologists      1034
Homeopath            843
Ayurveda             786
ENT Specialist       728
Name: Profile, dtype: int64

In [325]:
den=train1[train1.Profile=='Dentist'].Qualification.str.extract(r'(BDS, MDS) ?')
dent=test1[test1.Profile=='Dentist'].Qualification.str.extract(r'(BDS, MDS) ?')

In [326]:
den.fillna('BDS',inplace=True)
dent.fillna('BDS',inplace=True)

In [327]:
den.count() ############ Dentist degree done

0    1397
dtype: int64

In [328]:
den.rename(columns={0:'Dentist Degree'},inplace=True)
dent.rename(columns={0:'Dentist Degree'},inplace=True)

In [329]:
hom=train1[train1.Profile=='Homeopath'].Qualification.str.extract(r'(BHMS, MD)?')
homt=test1[test1.Profile=='Homeopath'].Qualification.str.extract(r'(BHMS, MD)?')

In [330]:
hom.fillna('BHMS',inplace=True)
homt.fillna('BHMS',inplace=True)

In [331]:
hom.rename(columns={0:'Homeopath'},inplace=True)
homt.rename(columns={0:'Homeopath'},inplace=True)

In [332]:
hom.count() ########### Homeopath done

Homeopath    843
dtype: int64

In [333]:
ayu=train1[train1.Profile=='Ayurveda'].Qualification.str.extract(r'(BAMS, MD) ?')
ayut=test1[test1.Profile=='Ayurveda'].Qualification.str.extract(r'(BAMS, MD) ?')

In [334]:
ayu.fillna('BAMS',inplace=True)
ayut.fillna('BAMS',inplace=True)

In [335]:
ayu.rename(columns={0:'Ayurveda'},inplace=True)
ayut.rename(columns={0:'Ayurveda'},inplace=True)

In [336]:
train1[train1.Profile=='Ayurveda'].Qualification.count()
test1[test1.Profile=='Ayurveda'].Qualification.count()

251

In [337]:
ent=train1[train1.Profile=='ENT Specialist'].Qualification.str.extract(r'(MBBS, MS) ?')
entt=test1[test1.Profile=='ENT Specialist'].Qualification.str.extract(r'(MBBS, MS) ?')

In [338]:
ent.fillna('MBBS',inplace=True)
entt.fillna('MBBS',inplace=True)

In [339]:
ent.rename(columns={0:'ENT Specialist'},inplace=True)
entt.rename(columns={0:'ENT Specialist'},inplace=True)

In [340]:
ent.count()

ENT Specialist    728
dtype: int64

In [341]:
train1[train1.Profile=='ENT Specialist'].Qualification.count()

728

In [342]:
gm=train1[train1.Profile=='General Medicine'].Qualification.str.extract(r'(MBBS, MD) ?')
gmt=test1[test1.Profile=='General Medicine'].Qualification.str.extract(r'(MBBS, MD) ?')

In [343]:
gm.fillna('MBBS',inplace=True)
gmt.fillna('MBBS',inplace=True)

In [344]:
gm.rename(columns={0:'General Medicine'},inplace=True)
gmt.rename(columns={0:'General Medicine'},inplace=True)

In [345]:
gm.count()

General Medicine    1173
dtype: int64

In [346]:
der=train1[train1.Profile=='Dermatologists'].Qualification.str.extract(r'(MBBS, MD) ?')
dert=test1[test1.Profile=='Dermatologists'].Qualification.str.extract(r'(MBBS, MD) ?')

In [347]:
der.fillna('MBBS',inplace=True)
dert.fillna('MBBS',inplace=True)

In [348]:
der.rename(columns={0:'Dermatologist'},inplace=True)
dert.rename(columns={0:'Dermatologist'},inplace=True)

In [349]:
new=pd.concat([den,hom,ayu,ent,gm,der],axis=1)
newt=pd.concat([dent,homt,ayut,entt,gmt,dert],axis=1)

In [350]:
new.fillna('Not Acquired',inplace=True)
newt.fillna('Not Acquired',inplace=True)

In [351]:
train1=pd.concat([train1,new],axis=1)
test1=pd.concat([test1,newt],axis=1)

In [352]:
train1.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Dentist Degree,Homeopath,Ayurveda,ENT Specialist,General Medicine,Dermatologist
0,"BHMS, MD - Homeopathy",24 years experience,100,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,Not Acquired,"BHMS, MD",Not Acquired,Not Acquired,Not Acquired,Not Acquired
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,Not Acquired,Not Acquired,"BAMS, MD",Not Acquired,Not Acquired,Not Acquired
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300,Not Acquired,Not Acquired,Not Acquired,"MBBS, MS",Not Acquired,Not Acquired
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250,Not Acquired,Not Acquired,BAMS,Not Acquired,Not Acquired,Not Acquired
4,BAMS,20 years experience,100,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,Not Acquired,Not Acquired,BAMS,Not Acquired,Not Acquired,Not Acquired


In [353]:
train1['Place']=train1['Place'].str.extract(r'.* (.*)')
test1['Place']=test1['Place'].str.extract(r'.* (.*)')

In [354]:
train1.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Dentist Degree,Homeopath,Ayurveda,ENT Specialist,General Medicine,Dermatologist
0,"BHMS, MD - Homeopathy",24 years experience,100,Ernakulam,Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100,Not Acquired,"BHMS, MD",Not Acquired,Not Acquired,Not Acquired,Not Acquired
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98,Bangalore,Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350,Not Acquired,Not Acquired,"BAMS, MD",Not Acquired,Not Acquired,Not Acquired
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,Bangalore,ENT Specialist,NaN,300,Not Acquired,Not Acquired,Not Acquired,"MBBS, MS",Not Acquired,Not Acquired
3,"BSc - Zoology, BAMS",12 years experience,NaN,Bangalore,Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250,Not Acquired,Not Acquired,BAMS,Not Acquired,Not Acquired,Not Acquired
4,BAMS,20 years experience,100,Chennai,Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250,Not Acquired,Not Acquired,BAMS,Not Acquired,Not Acquired,Not Acquired


In [355]:
train1['Experience']=train1['Experience'].str.extract(r'(^\d{1,2}) .*')
test1['Experience']=test1['Experience'].str.extract(r'(^\d{1,2}) .*')

In [356]:
test1['Place'].fillna('Bangalore',inplace=True)

In [357]:
train1['Place'].fillna('Mumbai',inplace=True)

In [358]:
train1[train1.Miscellaneous_Info.isnull()].head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees,Dentist Degree,Homeopath,Ayurveda,ENT Specialist,General Medicine,Dermatologist
2,"MBBS, MS - Otorhinolaryngology",9,NaN,Bangalore,ENT Specialist,NaN,300,Not Acquired,Not Acquired,Not Acquired,"MBBS, MS",Not Acquired,Not Acquired
5,BAMS,8,NaN,Chennai,Ayurveda,NaN,100,Not Acquired,Not Acquired,BAMS,Not Acquired,Not Acquired,Not Acquired
6,BHMS,42,NaN,Delhi,Homeopath,NaN,200,Not Acquired,BHMS,Not Acquired,Not Acquired,Not Acquired,Not Acquired
8,"MBBS, MD - General Medicine",14,NaN,Hyderabad,General Medicine,NaN,100,Not Acquired,Not Acquired,Not Acquired,Not Acquired,"MBBS, MD",Not Acquired
9,"BSc, BDS",23,NaN,Ernakulam,Dentist,NaN,100,BDS,Not Acquired,Not Acquired,Not Acquired,Not Acquired,Not Acquired


In [359]:
train1.isnull().sum()

Qualification            0
Experience               0
Rating                3302
Place                    0
Profile                  0
Miscellaneous_Info    2620
Fees                     0
Dentist Degree           0
Homeopath                0
Ayurveda                 0
ENT Specialist           0
General Medicine         0
Dermatologist            0
dtype: int64

In [360]:
train1.drop(columns=['Qualification','Rating','Miscellaneous_Info'],inplace=True)

In [361]:
test1.drop(columns=['Qualification','Rating','Miscellaneous_Info'],inplace=True)

In [362]:
le=LabelEncoder()
train1['Place']=le.fit_transform(train1['Place'])
train1['Profile']=le.fit_transform(train1['Profile'])
test1['Place']=le.fit_transform(test1['Place'])
test1['Profile']=le.fit_transform(test1['Profile'])

In [363]:
train1['Dentist Degree']=le.fit_transform(train1['Dentist Degree'])
train1['Homeopath']=le.fit_transform(train1['Homeopath'])
train1['Ayurveda']=le.fit_transform(train1['Ayurveda'])
train1['ENT Specialist']=le.fit_transform(train1['ENT Specialist'])
train1['General Medicine']=le.fit_transform(train1['General Medicine'])
train1['Dermatologist']=le.fit_transform(train1['Dermatologist'])

In [364]:
test1['Dentist Degree']=le.fit_transform(test1['Dentist Degree'])
test1['Homeopath']=le.fit_transform(test1['Homeopath'])
test1['Ayurveda']=le.fit_transform(test1['Ayurveda'])
test1['ENT Specialist']=le.fit_transform(test1['ENT Specialist'])
test1['General Medicine']=le.fit_transform(test1['General Medicine'])
test1['Dermatologist']=le.fit_transform(test1['Dermatologist'])

In [365]:
label=train1['Fees']
train1.drop(columns=['Fees'],inplace=True)

In [366]:
stsc=StandardScaler()
train1['Experience']=stsc.fit_transform(train1['Experience'].values.reshape(-1,1))
test1['Experience']=stsc.fit_transform(test1['Experience'].values.reshape(-1,1))

E:\New folder\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
E:\New folder\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
E:\New folder\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
E:\New folder\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [367]:
train1['Experience'].std()

1.0000838890987578

In [410]:
train1.head()

,Experience,Place,Profile,Dentist Degree,Homeopath,Ayurveda,ENT Specialist,General Medicine,Dermatologist
0,0.600979,4,5,2,1,2,2,2,2
1,-0.476040,0,0,2,2,1,2,2,2
2,-0.745295,0,3,2,2,2,1,2,2
3,-0.476040,0,0,2,2,0,2,2,2
4,0.241972,1,0,2,2,0,2,2,2


In [411]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5961 entries, 0 to 5960
Data columns (total 9 columns):
Experience          5961 non-null float64
Place               5961 non-null int32
Profile             5961 non-null int32
Dentist Degree      5961 non-null int32
Homeopath           5961 non-null int32
Ayurveda            5961 non-null int32
ENT Specialist      5961 non-null int32
General Medicine    5961 non-null int32
Dermatologist       5961 non-null int32
dtypes: float64(1), int32(8)
memory usage: 232.9 KB


In [368]:
def metric(y_act, y_pred):
    return(1-(np.sqrt(np.mean((np.log1p(y_act) - np.log1p(y_pred))**2))))

from sklearn.metrics import make_scorer
my_metric = make_scorer(metric, greater_is_better=False)

In [369]:
xtrain,xtest,ytrain,ytest=train_test_split(train1,label,test_size=0.2,random_state=50)

In [370]:
lin_reg=LinearRegression()
log_reg=LogisticRegression()
dst_reg=DecisionTreeRegressor()
rnd_reg=RandomForestRegressor()
ada_reg=AdaBoostRegressor(dst_reg)
ext_reg=ExtraTreesRegressor()
svm=SVC()

In [371]:
lin_reg.fit(xtrain,ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [372]:
lin_pred=lin_reg.predict(xtest)

In [373]:
mse_lin=np.sqrt(mean_squared_log_error(ytest,lin_pred))

In [374]:
1-mse_lin

0.3373525695024857

In [375]:
cv_lin=cross_val_score(lin_reg,xtrain,ytrain,scoring= my_metric,cv=5)

In [376]:
cv_lin

array([-0.30845076, -0.30910129, -0.31935494, -0.3337995 , -0.35401955])

In [377]:
cv_lin_sc=cv_lin.mean()

In [378]:
1-np.sqrt(-cv_lin_sc)

0.4299603451015831

In [379]:
log_reg.fit(xtrain,ytrain)

E:\New folder\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\New folder\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [380]:
log_pred=log_reg.predict(xtest)

In [381]:
mse_log=mean_squared_log_error(ytest,log_pred)

In [382]:
1-mse_log

0.1998854556876749

In [383]:
dst_reg.fit(xtrain,ytrain)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [384]:
dst_pred=dst_reg.predict(xtest)

In [385]:
1-np.sqrt(mean_squared_log_error(ytest,dst_pred))

0.2580679852398782

In [386]:
1-np.sqrt(-cross_val_score(dst_reg,xtrain,ytrain,scoring=my_metric,cv=5)).mean()

0.5186861096920302

In [387]:
rnd_reg.fit(xtrain,ytrain)

E:\New folder\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [388]:
rnd_pred=rnd_reg.predict(xtest)

In [389]:
1-np.sqrt(mean_squared_log_error(ytest,rnd_pred))

0.3163020423291041

In [390]:
1-np.sqrt(-cross_val_score(rnd_reg,xtrain,ytrain,scoring=my_metric,cv=5)).mean()

0.46136279651167633

In [391]:
ada_reg.fit(xtrain,ytrain)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=50,
         random_state=None)

In [392]:
ada_pred=ada_reg.predict(xtest)

In [393]:
1-np.sqrt(mean_squared_log_error(ytest,ada_pred))

0.28556232959781813

In [394]:
1-np.sqrt(-cross_val_score(ada_reg,xtrain,ytrain,scoring=my_metric,cv=5)).mean()

0.46835704330603767

In [395]:
ext_reg.fit(xtrain,ytrain)

E:\New folder\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False)

In [396]:
ext_pred=ext_reg.predict(xtest)

In [397]:
1-np.sqrt(mean_squared_log_error(ytest,ext_pred))

0.2774848741879671

In [398]:
1-np.sqrt(-cross_val_score(ext_reg,xtrain,ytrain,scoring=my_metric,cv=5)).mean()

0.4913922899681239

In [399]:
svm.fit(xtrain,ytrain)

E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [400]:
svm_pred=svm.predict(xtest)

In [401]:
svm_pred=mean_squared_log_error(ytest,svm_pred)

In [402]:
1-svm_pred

0.3085854052851945

In [403]:
1-np.sqrt(-cross_val_score(svm,xtrain,ytrain,scoring=my_metric,cv=5)).mean()

E:\New folder\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
E:\New folder\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in vers

0.6272735719087144

In [404]:
train1.isnull().sum()

Experience          0
Place               0
Profile             0
Dentist Degree      0
Homeopath           0
Ayurveda            0
ENT Specialist      0
General Medicine    0
Dermatologist       0
dtype: int64

In [405]:
test=dst_reg.predict(test1)

In [406]:
test=list(test)

In [407]:
test=pd.DataFrame(test)

In [408]:
test.rename(columns={0:'Fees'},inplace=True)

In [409]:
test.to_excel('Submit_dst.xlsx')